In [2]:
import os, sys
sys.path.insert(0, '..')

import matplotlib.pyplot as plt
import torch.nn.functional as F
import node.node_lib as node
import torch, torch.nn as nn
import pandas as pd
import numpy as np
import time
import re

%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline

from common import data_load_home_credit, find_optimal_cutoff, plot_confusion_matrix, plot_auc

device = 'cuda' if torch.cuda.is_available() else 'cpu'
experiment_name = 'year_node_8layers'
experiment_name = '{}_{}.{:0>2d}.{:0>2d}_{:0>2d}:{:0>2d}'.format(experiment_name, *time.gmtime()[:5])
print("experiment:", experiment_name)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: CUDA_VISIBLE_DEVICES=0
experiment: year_node_8layers_2021.07.06_12:26


In [3]:
train, test = data_load_home_credit('/media/ismaeel/Work/msds19029_thesis/dataset/home_with_missing.csv')

In [5]:
feats = [f for f in train.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]

cat_features = [
    'CODE_GENDER', 'FLAG_OWN_CAR', 'NAME_CONTRACT_TYPE', 'NAME_EDUCATION_TYPE',
    'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'NAME_INCOME_TYPE', 'OCCUPATION_TYPE',
    'ORGANIZATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_TYPE_SUITE', 'WALLSMATERIAL_MODE']

In [7]:
in_features = (train.shape[1] - 1)
X_train = train[feats]
Y_train = train['TARGET']

X_test = test[feats]
Y_test = test['TARGET']

In [11]:
X_train.shape

(239818, 254)

In [13]:
model = nn.Sequential(
    node.DenseBlock(in_features, 
                   128, 
                   num_layers=8, 
                   tree_dim=3, 
                   depth=6, 
                   flatten_output=False,
                   choice_function=node.entmax15, 
                   bin_function=node.entmoid15),
                   node.Lambda(lambda x: x[..., 0].mean(dim=-1)),  # average first channels of every tree
    
).to(device)

with torch.no_grad():
    res = model(torch.as_tensor(X_train.values, device=device))
    
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

RuntimeError: einsum(): operands do not broadcast with remapped shapes [original->remapped]: [239818, 254]->[239818, 1, 1, 254] [255, 128, 6]->[1, 128, 6, 255]